In [ ]:
from __future__ import (absolute_import, division, print_function)
from functools import reduce, partial
from operator import mul
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from pyneqsys.symbolic import SymbolicSys, TransformedSys, linear_exprs
sp.init_printing()
prod = lambda x: reduce(mul, x)
print(sp.__version__)

Let's consider:
$$
H^+ + OH^- \leftrightharpoons H_2O \\
NH_4^+ \leftrightharpoons H^+ + NH_3
$$

In [ ]:
names = 'H+ OH- NH4+ NH3 H2O'.split()
NH3_idx = 3
NH3_varied = np.logspace(-7, 0)
c0 = 1e-7, 1e-7, 1e-7, 1, 55
K = Kw, Ka = 10**-14/55, 10**-9.24

In [ ]:
stoichs = [[1, 1, 0, 0, -1], [1, 0, -1, 1, 0]]
H = [1, 1, 4, 3, 2]
N = [0, 0, 1, 1, 0]
O = [0, 1, 0, 0, 1]
q = [1, -1, 1, 0, 0]
preserv = [H, N, O, q]
def get_f(x, params, backend, lnK):
    init_concs = params[:5]
    eq_constants = params[5:]
    le = linear_exprs(preserv, x, linear_exprs(preserv, init_concs), rref=True)
    if lnK:
        return le + [
            sum(backend.log(xi)*p for xi, p in zip(x, coeffs)) - backend.log(K) 
            for coeffs, K in zip(stoichs, eq_constants)
        ]
    else:
        return le + [
            prod(xi**p for xi, p in zip(x, coeffs)) - K for coeffs, K in zip(stoichs, eq_constants)
        ]

In [ ]:
neqsys = SymbolicSys.from_callback(partial(get_f, lnK=False), 5, 7, names=names)
neqsys.exprs

In [ ]:
neqsys.get_jac()

In [ ]:
%matplotlib inline
def solve_and_plot(nsys):
    fig = plt.figure(figsize=(12, 4))
    ax_out = plt.subplot(1, 2, 1, xscale='log', yscale='log')
    ax_err = plt.subplot(1, 2, 2, xscale='log')
    ax_err.set_yscale('symlog', linthreshy=1e-14)
    xres, info_dicts = nsys.solve_and_plot_series(c0, c0+K, NH3_varied, NH3_idx, 'scipy', 
                                                  plot_kwargs=dict(ax=ax_out),
                                                  plot_residuals_kwargs=dict(ax=ax_err))
    for ax in (ax_out, ax_err):
        ax.set_xlabel('[NH3]0 / M')
    ax_out.set_ylabel('Concentration / M')
    ax_out.legend(loc='best')
    ax_err.set_ylabel('Residuals')
    
    avg_nfev = np.average([nfo['nfev'] for nfo in info_dicts])
    avg_njev = np.average([nfo['njev'] for nfo in info_dicts])
    success = np.average([int(nfo['success']) for nfo in info_dicts])
    return {'avg_nfev': avg_nfev, 'avg_njev': avg_njev, 'success': success}

    
solve_and_plot(neqsys)

Now let's see how pyneqsys can transform our system:

In [ ]:
tneqsys = TransformedSys.from_callback(partial(get_f, lnK=True), (sp.exp, sp.log), 5, 7)
tneqsys.exprs

In [ ]:
c_res, info = tneqsys.solve([1]*5, np.array(c0+K))
c0, c_res, info['success']

In [ ]:
solve_and_plot(tneqsys)